In [52]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv
/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv
/kaggle/input/llm-detect-ai-generated-text/test_essays.csv
/kaggle/input/llm-detect-ai-generated-text/train_essays.csv
/kaggle/input/training-dataset/Generated_text.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v2.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts_v2.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_7_prompts.csv
/kaggle/input/llm-7-prompt-training-dataset/train_essays_RDizzl3_seven_v1.csv


In [53]:
import pandas as pd
from collections import defaultdict, Counter
import re
from math import log
from math import exp
from math import log, exp
import csv
from collections import defaultdict
from math import log, exp
import re
import re
from collections import defaultdict
from math import log, exp
from sklearn.metrics import accuracy_score

In [54]:
training_data  = pd.read_csv('/kaggle/input/training-dataset/Generated_text.csv')
training_dataset  = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv').to_dict('records')
test_data  = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
train_prompts_data   = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_prompts.csv')

In [55]:
train1 = training_data[training_data.RDizzl3_seven == False].reset_index(drop=True)
train1=training_data[training_data["label"]==1].sample(8000)
train = training_data[training_data.RDizzl3_seven == True].reset_index(drop=True)

In [56]:
train=pd.concat([train,train1])

In [57]:
train_ratio = 0.8
train_size = int(len(training_data) * train_ratio)

train_dataset = training_data[:train_size]
dev_dataset = training_data[train_size:]

In [58]:
text = train_dataset['text']
labels = train_dataset['label']

In [59]:
from collections import Counter

def build_vocabulary(data, min_occurrences=5):
    all_words = [word for essay in data['text'] for word in essay.split()]
    word_counts = Counter(all_words)
    
    # Choose words that occur less than min_occurrences times
    rare_words = [word for word, count in word_counts.items() if count < min_occurrences]
    
    # Exclude rare words from the vocabulary
    vocabulary = list(set(all_words) - set(rare_words))
    
    reverse_index = {word: index for index, word in enumerate(vocabulary)}
    
    return vocabulary, reverse_index

vocabulary, reverse_index = build_vocabulary(train)


In [60]:
def train_naive_bayes_classifier(documents, labels, alpha):
    class_probs = defaultdict(float)
    word_probs = defaultdict(lambda: defaultdict(float))
    classes = set()

    class_counts = defaultdict(int)
    for label in labels:
        class_counts[label] += 1
        classes.add(label)

    total_documents = len(labels)

    for label, count in class_counts.items():
        class_probs[label] = count / total_documents

    word_counts = defaultdict(lambda: defaultdict(int))

    for doc, label in zip(documents, labels):
        words = re.findall(r'\b\w+\b', doc.lower())
        for word in words:
            word_counts[label][word] += 1

    for label in classes:
        total_words_in_class = sum(word_counts[label].values())
        total_unique_words = len(set(word_counts[label].keys()))

        for word, count in word_counts[label].items():
            word_probs[label][word] = (count + alpha) / (total_words_in_class + alpha * total_unique_words)

    return class_probs, word_probs, classes

def predict_naive_bayes_classifier(documents, class_probs, word_probs, classes):
    predictions = []

    for document in documents:
        words = re.findall(r'\b\w+\b', str(document).lower())
        scores = defaultdict(float)

        for label in classes:
            scores[label] = log(class_probs[label])

            for word in words:
                scores[label] += log(word_probs[label].get(word, 1e-10))

        exp_scores = {label: exp(score) for label, score in scores.items()}
        sum_exp_scores = sum(exp_scores.values())

        if sum_exp_scores == 0:
            probabilities = {label: 1 / len(classes) for label in classes}
        else:
            probabilities = {label: exp_score / sum_exp_scores for label, exp_score in exp_scores.items()}

        predictions.append(probabilities)

    return predictions



In [61]:
dev_data_text = dev_dataset['text']
dev_data_labels = dev_dataset['label']

In [62]:
alpha_values_to_try = [0.01, 0.1, 0.5, 1.0, 5.0, 10.0]
for alpha in alpha_values_to_try:
    class_probs, word_probs, classes = train_naive_bayes_classifier(text, labels, alpha)
    predictions = predict_naive_bayes_classifier(dev_data_text, class_probs, word_probs, classes)
    result=[]
    for i in range(len(predictions)):
        if(predictions[i][0] >= predictions[i][1]):
            result.append(0)
        else:
            result.append(1)
    count = 0
    for i in range(len(result)):
        if(result[i] == dev_data_labels.iloc[i]):
            count = count + 1

    accuracy = count / len(result)
    print("For alpha value:", alpha, "Accuracy:", accuracy)

For alpha value: 0.01 Accuracy: 0.1586806329396033
For alpha value: 0.1 Accuracy: 0.1586806329396033
For alpha value: 0.5 Accuracy: 0.1586806329396033
For alpha value: 1.0 Accuracy: 0.1586806329396033
For alpha value: 5.0 Accuracy: 0.15856919991085358
For alpha value: 10.0 Accuracy: 0.15856919991085358


In [64]:
from collections import Counter

def calculate_probabilities(data, vocabulary):
    # Calculate occurrence probability
    total_documents = len(data)
    word_counts = Counter(' '.join([row['text'] for row in data]).split())
    occurrence_probability = {word: word_counts[word] / total_documents for word in vocabulary}

    # Calculate conditional probability
    llm_documents = [row['text'] for row in data if row['generated'] == 1]
    llm_word_counts = Counter(' '.join(llm_documents).split())
    conditional_probability_llm = {word: llm_word_counts[word] / len(llm_documents) for word in vocabulary}

    return occurrence_probability, conditional_probability_llm



In [65]:
occurrence_probability, conditional_probability_llm = calculate_probabilities(training_dataset, vocabulary)

In [67]:
# Derive Top 10 words that predict each class
top_10_llm_words = sorted(vocabulary, key=lambda word: conditional_probability_llm[word], reverse=True)[:10]
top_10_human_words = sorted(vocabulary, key=lambda word: occurrence_probability[word], reverse=True)[:10]

print("Top 10 words predicting LLM essays:", top_10_llm_words)
print("Top 10 words predicting human essays:", top_10_human_words)

Top 10 words predicting LLM essays: ['the', 'of', 'to', 'for', 'a', 'and', 'that', 'votes', 'is', 'in']
Top 10 words predicting human essays: ['the', 'to', 'of', 'a', 'and', 'in', 'is', 'that', 'for', 'are']


In [71]:
test_text = test_data['text']
print(test_text)
testing_data_probabilities = predict_naive_bayes_classifier(test_text, class_probs, word_probs, classes)
print(testing_data_probabilities)

0    Aaa bbb ccc.
1    Bbb ccc ddd.
2    CCC ddd eee.
Name: text, dtype: object
[{0: 0.6704230005462533, 1: 0.3295769994537467}, {0: 0.7242436061737354, 1: 0.2757563938262646}, {0: 0.7242436061737354, 1: 0.2757563938262646}]


In [73]:
final_testing_output = []
for i in testing_data_probabilities:
    final_testing_output.append(i[1])
predicted_output = np.array(final_testing_output)

In [76]:
submission= pd.DataFrame({'id':test_data["id"],'generated':predicted_output})
submission.to_csv('submission.csv', index=False)
print(submission)

         id  generated
0  0000aaaa   0.329577
1  1111bbbb   0.275756
2  2222cccc   0.275756
